In [18]:
import os
import numpy as np
import pandas as pd
import xarray as xr
import tempfile
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from datetime import datetime


In [19]:
from station.data_submission import DataSubmission
from station.station import StationData


from station.dat_to_nc_all_var import DatToNcAllVar


In [20]:
# Initialize the converter
converter = DatToNcAllVar(
    name="Vienna_AllVar",
    directory="measurements/Vienna",
    target_directory="station_data_as_nc",
    hourly=True,
    keep_original=True
)


In [21]:
converter.execute()

Extracting Vienna_AllVar...


100%|██████████| 3714/3714 [00:30<00:00, 123.56it/s]


Saving to station_data_as_nc/vienna_allvar.nc


/Users/ferdinand/Desktop/MasterThesis/digital-twin-for-weather-station/station/dat_to_nc_all_var.py:309: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  self.dataframe[col] = pd.to_numeric(self.dataframe[col], errors='ignore')


In [25]:
raw_df = converter.original_df

print(raw_df.head())

processed_df = converter.dataframe

print(processed_df.head())


                     bmp180_temp  bmp180_pres  bmp180_slp  bmp180_alt  \
datetime                                                                
2017-04-27 00:00:00      -999.99      -999.99     -999.99     -999.99   
2017-04-27 00:01:00      -999.99      -999.99     -999.99     -999.99   
2017-04-27 00:02:00      -999.99      -999.99     -999.99     -999.99   
2017-04-27 00:03:00      -999.99      -999.99     -999.99     -999.99   
2017-04-27 00:04:00      -999.99      -999.99     -999.99     -999.99   

                     bmp280_temp  bmp280_pres  bmp280_slp  bmp280_alt  \
datetime                                                                
2017-04-27 00:00:00         8.38       993.06     1012.37       159.0   
2017-04-27 00:01:00         8.37       993.08     1012.40       159.0   
2017-04-27 00:02:00         8.36       993.03     1012.35       159.0   
2017-04-27 00:03:00         8.35       993.02     1012.34       159.0   
2017-04-27 00:04:00         8.33       993.04     

In [26]:
raw_df.to_csv("station_data_raw_original/vienna_raw.csv")

In [27]:
processed_df.to_csv("station_data_raw_original/vienna_processed.csv")

In [68]:
def counting_tipping_values(station_data, raworprocessed):
    # Count the number of times tipping is 0.0
    tipping_zero_count = station_data[station_data['tipping'] == 0.0].shape[0]

    # Count the number of times tipping is non-zero and not NaN
    tipping_non_zero_count = station_data[(station_data['tipping'] != 0.0) & (station_data['tipping'].notna())].shape[0]

    # Total count of rows where tipping is not NaN
    tipping_total_count = station_data[station_data['tipping'].notna()].shape[0]

    # Print the counts
    print("Raw or processed:", raworprocessed)
    print(f"Tipping equals 0.0: {tipping_zero_count}")
    print(f"Tipping is non-zero and not NaN: {tipping_non_zero_count}")
    print(f"Total number of tipping measurements (non-NaN): {tipping_total_count}")

    # Calculate percentages
    tipping_zero_percentage = (tipping_zero_count / tipping_total_count) * 100
    tipping_non_zero_percentage = (tipping_non_zero_count / tipping_total_count) * 100

    print(f"Percentage of tipping values equal to 0.0: {tipping_zero_percentage:.2f}%")
    print(f"Percentage of tipping values non-zero: {tipping_non_zero_percentage:.2f}%")

# Marshall

## Basic

In [69]:
marshall_raw_df = pd.read_csv("station_data_raw_original/marshall_raw.csv", index_col=0)
marshall_processed_df = pd.read_csv("station_data_raw_original/marshall_processed.csv", index_col=0)

In [70]:
marshall_processed_df.describe()


,bmp180_temp,bmp180_pres,bmp180_slp,bmp180_alt,bmp280_temp,bmp280_pres,bmp280_slp,bmp280_alt,bme_temp,bme_pres,...,htu_temp,htu_hum,mcp9808,tipping,vis_light,ir_light,uv_light,wind_dir,wind_speed,tas
count,0.0,0.0,0.0,0.0,17070.000000,17072.000000,17072.000000,0.0,0.0,0.0,...,27336.000000,29608.000000,12585.000000,32076.000000,18879.000000,18506.000000,10961.000000,30829.000000,16382.000000,30578.000000
mean,NaN,NaN,NaN,NaN,13.291198,996.852766,1015.925743,NaN,NaN,NaN,...,13.143690,68.725581,12.045101,0.074629,344.470470,1133.584243,79.520208,209.613783,0.849525,285.976937
std,NaN,NaN,NaN,NaN,8.730016,21.877956,22.364367,NaN,NaN,NaN,...,8.538697,17.703144,8.926236,2.801776,158.352829,1670.567697,99.160300,105.766542,0.633719,8.628116
min,NaN,NaN,NaN,NaN,-10.915000,600.490000,611.470000,NaN,NaN,NaN,...,-11.000000,12.100000,-11.200000,0.000000,0.000000,0.000000,0.000000,0.013044,0.001500,262.106667
25%,NaN,NaN,NaN,NaN,6.090000,993.315000,1012.130000,NaN,NaN,NaN,...,6.200000,56.300000,4.650000,0.000000,261.000000,253.000000,12.000000,122.217641,0.348042,278.845000
50%,NaN,NaN,NaN,NaN,13.082500,998.255000,1017.150000,NaN,NaN,NaN,...,12.850000,70.300000,11.200000,0.000000,264.500000,303.500000,36.000000,230.813037,0.750000,285.550000
75%,NaN,NaN,NaN,NaN,20.040000,1003.010000,1022.270000,NaN,NaN,NaN,...,19.600000,82.750000,19.100000,0.000000,342.500000,1079.750000,110.000000,299.266364,1.224407,292.771875
max,NaN,NaN,NaN,NaN,37.910000,1023.065000,1043.630000,NaN,NaN,NaN,...,37.700000,99.800000,37.100000,354.400000,1094.000000,8779.000000,448.000000,359.995453,4.215000,310.721667


In [71]:
marshall_raw_df.describe()

,bmp180_temp,bmp180_pres,bmp180_slp,bmp180_alt,bmp280_temp,bmp280_pres,bmp280_slp,bmp280_alt,bme_temp,bme_pres,...,bme_hum,htu_temp,htu_hum,mcp9808,tipping,vis_light,ir_light,uv_light,wind_dir,wind_speed
count,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,...,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06
mean,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-5.688633e+02,-1.651387e+02,-1.573345e+02,-5.089579e+02,-9.999900e+02,-9.999900e+02,...,-9.999900e+02,-3.100451e+02,-2.883837e+02,-6.643431e+02,-2.800229e+02,-4.224009e+02,-9.368240e+01,-5.484988e+02,-1.487072e+02,-6.289503e+02
std,4.661161e-12,4.661161e-12,4.661161e-12,4.661161e-12,5.015931e+02,9.735705e+02,9.827422e+02,5.727031e+02,4.661161e-12,4.661161e-12,...,4.661161e-12,4.717284e+02,5.043525e+02,4.761447e+02,4.490081e+02,6.718095e+02,1.513093e+03,5.212565e+02,5.397940e+02,4.834010e+02
min,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,...,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02
25%,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,...,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02
50%,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,...,-9.999900e+02,5.800000e+00,5.620000e+01,-9.999900e+02,0.000000e+00,-9.999900e+02,-9.999900e+02,-9.999900e+02,1.128000e+02,-9.999900e+02
75%,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,1.163000e+01,9.956200e+02,1.014450e+03,1.590000e+02,-9.999900e+02,-9.999900e+02,...,-9.999900e+02,1.550000e+01,7.630000e+01,3.500000e+00,0.000000e+00,2.620000e+02,2.540000e+02,3.000000e+00,2.565000e+02,3.600000e-01
max,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,8.147000e+01,1.087580e+03,1.107610e+03,1.590000e+02,-9.999900e+02,-9.999900e+02,...,-9.999900e+02,8.140000e+01,9.990000e+01,1.247000e+02,2.800000e+01,1.630000e+03,1.354100e+04,7.360000e+02,3.600000e+02,9.824000e+01


In [72]:
print(marshall_processed_df.info())
print(marshall_raw_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 44554 entries, 2017-04-24 07:00:00 to 2022-05-23 23:00:00
Data columns (total 23 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   bmp180_temp  0 non-null      float64
 1   bmp180_pres  0 non-null      float64
 2   bmp180_slp   0 non-null      float64
 3   bmp180_alt   0 non-null      float64
 4   bmp280_temp  17070 non-null  float64
 5   bmp280_pres  17072 non-null  float64
 6   bmp280_slp   17072 non-null  float64
 7   bmp280_alt   0 non-null      float64
 8   bme_temp     0 non-null      float64
 9   bme_pres     0 non-null      float64
 10  bme_slp      0 non-null      float64
 11  bme_alt      0 non-null      float64
 12  bme_hum      0 non-null      float64
 13  htu_temp     27336 non-null  float64
 14  htu_hum      29608 non-null  float64
 15  mcp9808      12585 non-null  float64
 16  tipping      32076 non-null  float64
 17  vis_light    18879 non-null  float64
 18  ir_light     18506 

In [73]:
counting_tipping_values(marshall_raw_df, "raw")

Raw or processed: raw
Tipping equals 0.0: 1918917
Tipping is non-zero and not NaN: 754290
Total number of tipping measurements (non-NaN): 2673207
Percentage of tipping values equal to 0.0: 71.78%
Percentage of tipping values non-zero: 28.22%


In [74]:
counting_tipping_values(marshall_processed_df, "processed")

Raw or processed: processed
Tipping equals 0.0: 30236
Tipping is non-zero and not NaN: 1840
Total number of tipping measurements (non-NaN): 32076
Percentage of tipping values equal to 0.0: 94.26%
Percentage of tipping values non-zero: 5.74%


# Vienna

## Basic

In [75]:
vienna_raw_df = pd.read_csv("station_data_raw_original/vienna_raw.csv", index_col=0)
vienna_processed_df = pd.read_csv("station_data_raw_original/vienna_processed.csv", index_col=0)

In [76]:
vienna_processed_df.describe()

,bmp180_temp,bmp180_pres,bmp180_slp,bmp180_alt,bmp280_temp,bmp280_pres,bmp280_slp,bmp280_alt,bme_temp,bme_pres,...,htu_temp,htu_hum,mcp9808,tipping,vis_light,ir_light,uv_light,wind_dir,wind_speed,tas
count,0.0,0.0,0.0,0.0,17070.000000,17072.000000,17072.000000,0.0,0.0,0.0,...,27336.000000,29608.000000,12585.000000,32076.000000,18879.000000,18506.000000,10961.000000,30829.000000,16382.000000,30578.000000
mean,NaN,NaN,NaN,NaN,13.291198,996.852766,1015.925743,NaN,NaN,NaN,...,13.143690,68.725581,12.045101,0.074629,344.470470,1133.584243,79.520208,209.613783,0.849525,285.976937
std,NaN,NaN,NaN,NaN,8.730016,21.877956,22.364367,NaN,NaN,NaN,...,8.538697,17.703144,8.926236,2.801776,158.352829,1670.567697,99.160300,105.766542,0.633719,8.628116
min,NaN,NaN,NaN,NaN,-10.915000,600.490000,611.470000,NaN,NaN,NaN,...,-11.000000,12.100000,-11.200000,0.000000,0.000000,0.000000,0.000000,0.013044,0.001500,262.106667
25%,NaN,NaN,NaN,NaN,6.090000,993.315000,1012.130000,NaN,NaN,NaN,...,6.200000,56.300000,4.650000,0.000000,261.000000,253.000000,12.000000,122.217641,0.348042,278.845000
50%,NaN,NaN,NaN,NaN,13.082500,998.255000,1017.150000,NaN,NaN,NaN,...,12.850000,70.300000,11.200000,0.000000,264.500000,303.500000,36.000000,230.813037,0.750000,285.550000
75%,NaN,NaN,NaN,NaN,20.040000,1003.010000,1022.270000,NaN,NaN,NaN,...,19.600000,82.750000,19.100000,0.000000,342.500000,1079.750000,110.000000,299.266364,1.224407,292.771875
max,NaN,NaN,NaN,NaN,37.910000,1023.065000,1043.630000,NaN,NaN,NaN,...,37.700000,99.800000,37.100000,354.400000,1094.000000,8779.000000,448.000000,359.995453,4.215000,310.721667


In [77]:
vienna_raw_df.describe()

,bmp180_temp,bmp180_pres,bmp180_slp,bmp180_alt,bmp280_temp,bmp280_pres,bmp280_slp,bmp280_alt,bme_temp,bme_pres,...,bme_hum,htu_temp,htu_hum,mcp9808,tipping,vis_light,ir_light,uv_light,wind_dir,wind_speed
count,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,...,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06,2.673207e+06
mean,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-5.688633e+02,-1.651387e+02,-1.573345e+02,-5.089579e+02,-9.999900e+02,-9.999900e+02,...,-9.999900e+02,-3.100451e+02,-2.883837e+02,-6.643431e+02,-2.800229e+02,-4.224009e+02,-9.368240e+01,-5.484988e+02,-1.487072e+02,-6.289503e+02
std,4.661161e-12,4.661161e-12,4.661161e-12,4.661161e-12,5.015931e+02,9.735705e+02,9.827422e+02,5.727031e+02,4.661161e-12,4.661161e-12,...,4.661161e-12,4.717284e+02,5.043525e+02,4.761447e+02,4.490081e+02,6.718095e+02,1.513093e+03,5.212565e+02,5.397940e+02,4.834010e+02
min,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,...,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02
25%,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,...,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02
50%,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,...,-9.999900e+02,5.800000e+00,5.620000e+01,-9.999900e+02,0.000000e+00,-9.999900e+02,-9.999900e+02,-9.999900e+02,1.128000e+02,-9.999900e+02
75%,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,1.163000e+01,9.956200e+02,1.014450e+03,1.590000e+02,-9.999900e+02,-9.999900e+02,...,-9.999900e+02,1.550000e+01,7.630000e+01,3.500000e+00,0.000000e+00,2.620000e+02,2.540000e+02,3.000000e+00,2.565000e+02,3.600000e-01
max,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,8.147000e+01,1.087580e+03,1.107610e+03,1.590000e+02,-9.999900e+02,-9.999900e+02,...,-9.999900e+02,8.140000e+01,9.990000e+01,1.247000e+02,2.800000e+01,1.630000e+03,1.354100e+04,7.360000e+02,3.600000e+02,9.824000e+01


In [78]:
print(vienna_processed_df.info())
print(vienna_raw_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 44554 entries, 2017-04-25 00:00:00 to 2022-05-23 23:00:00
Data columns (total 23 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   bmp180_temp  0 non-null      float64
 1   bmp180_pres  0 non-null      float64
 2   bmp180_slp   0 non-null      float64
 3   bmp180_alt   0 non-null      float64
 4   bmp280_temp  17070 non-null  float64
 5   bmp280_pres  17072 non-null  float64
 6   bmp280_slp   17072 non-null  float64
 7   bmp280_alt   0 non-null      float64
 8   bme_temp     0 non-null      float64
 9   bme_pres     0 non-null      float64
 10  bme_slp      0 non-null      float64
 11  bme_alt      0 non-null      float64
 12  bme_hum      0 non-null      float64
 13  htu_temp     27336 non-null  float64
 14  htu_hum      29608 non-null  float64
 15  mcp9808      12585 non-null  float64
 16  tipping      32076 non-null  float64
 17  vis_light    18879 non-null  float64
 18  ir_light     18506 

In [79]:
counting_tipping_values(vienna_raw_df, "raw")

Raw or processed: raw
Tipping equals 0.0: 1918917
Tipping is non-zero and not NaN: 754290
Total number of tipping measurements (non-NaN): 2673207
Percentage of tipping values equal to 0.0: 71.78%
Percentage of tipping values non-zero: 28.22%


In [80]:
counting_tipping_values(vienna_processed_df, "processed")

Raw or processed: processed
Tipping equals 0.0: 30236
Tipping is non-zero and not NaN: 1840
Total number of tipping measurements (non-NaN): 32076
Percentage of tipping values equal to 0.0: 94.26%
Percentage of tipping values non-zero: 5.74%


# Barbados

In [81]:
barbados_raw_df = pd.read_csv("station_data_raw_original/barbados_raw.csv", index_col=0)
barbados_processed_df = pd.read_csv("station_data_raw_original/barbados_processed.csv", index_col=0)


## Basic

In [82]:
barbados_processed_df.describe()

,bmp180_temp,bmp180_pres,bmp180_slp,bmp180_alt,bmp280_temp,bmp280_pres,bmp280_slp,bmp280_alt,bme_temp,bme_pres,...,htu_temp,htu_hum,mcp9808,tipping,vis_light,ir_light,uv_light,wind_dir,wind_speed,tas
count,0.0,0.0,0.0,0.0,19275.000000,19304.000000,19304.000000,0.0,0.0,0.0,...,19539.000000,15417.000000,17389.000000,29008.000000,22612.000000,21491.000000,13230.000000,0.0,26956.000000,24780.000000
mean,NaN,NaN,NaN,NaN,26.363526,984.414959,1015.581027,NaN,NaN,NaN,...,22.188743,85.960735,25.575539,0.193285,419.594529,1990.336373,147.866062,NaN,3.065972,297.753908
std,NaN,NaN,NaN,NaN,1.966420,11.590826,11.953170,NaN,NaN,NaN,...,6.201811,8.186707,2.048191,1.341190,228.350229,2400.179046,129.135906,NaN,2.429038,2.330054
min,NaN,NaN,NaN,NaN,-21.155000,648.600000,669.320000,NaN,NaN,NaN,...,1.500000,55.900000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.001000,251.995000
25%,NaN,NaN,NaN,NaN,24.975000,983.830000,1014.975000,NaN,NaN,NaN,...,21.900000,80.150000,24.200000,0.000000,260.000000,253.500000,29.000000,NaN,1.963417,296.561250
50%,NaN,NaN,NaN,NaN,26.160000,985.000000,1016.210000,NaN,NaN,NaN,...,24.400000,86.900000,25.400000,0.000000,266.500000,499.000000,124.500000,NaN,2.895667,297.826667
75%,NaN,NaN,NaN,NaN,27.660000,986.125000,1017.370000,NaN,NaN,NaN,...,26.200000,92.550000,27.100000,0.000000,536.625000,3262.250000,227.000000,NaN,3.824583,299.106667
max,NaN,NaN,NaN,NaN,33.410000,990.780000,1021.940000,NaN,NaN,NaN,...,31.100000,99.800000,32.200000,66.200000,1221.000000,10258.500000,516.500000,NaN,74.617241,303.870000


In [83]:
barbados_raw_df.describe()

,bmp180_temp,bmp180_pres,bmp180_slp,bmp180_alt,bmp280_temp,bmp280_pres,bmp280_slp,bmp280_alt,bme_temp,bme_pres,...,bme_hum,htu_temp,htu_hum,mcp9808,tipping,vis_light,ir_light,uv_light,wind_dir,wind_speed
count,1.817508e+06,1.817508e+06,1.817508e+06,1.817508e+06,1.817508e+06,1.817508e+06,1.817508e+06,1.817508e+06,1.817508e+06,1.817508e+06,...,1.817508e+06,1.817508e+06,1.817508e+06,1.817508e+06,1.817508e+06,1.817508e+06,1.817508e+06,1.817508e+06,1.817508e+06,1.817508e+06
mean,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-2.123054e+02,4.842709e+02,5.069669e+02,-2.188401e+01,-9.999900e+02,-9.999900e+02,...,-9.999900e+02,-1.967659e+02,-4.363836e+02,-2.603876e+02,-4.356546e+01,9.118536e+01,1.256991e+03,-1.646803e+02,-7.625920e+02,-4.196570e+01
std,2.501111e-12,2.501111e-12,2.501111e-12,2.501111e-12,4.332454e+02,8.225395e+02,8.353257e+02,5.379648e+02,2.501111e-12,2.501111e-12,...,2.501111e-12,4.197850e+02,5.426432e+02,4.592763e+02,2.041328e+02,6.374778e+02,2.569836e+03,4.744023e+02,4.254855e+02,2.072575e+02
min,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,...,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02
25%,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,2.312000e+01,6.486000e+02,6.693200e+02,2.740000e+02,-9.999900e+02,-9.999900e+02,...,-9.999900e+02,9.300000e+00,-9.999900e+02,-9.999900e+02,0.000000e+00,2.590000e+02,2.520000e+02,1.000000e+00,-9.999900e+02,1.240000e+00
50%,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,2.485000e+01,9.836200e+02,1.014750e+03,2.740000e+02,-9.999900e+02,-9.999900e+02,...,-9.999900e+02,2.350000e+01,6.970000e+01,2.420000e+01,0.000000e+00,2.610000e+02,2.540000e+02,2.000000e+00,-9.999900e+02,2.550000e+00
75%,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,2.670000e+01,9.854700e+02,1.016690e+03,2.740000e+02,-9.999900e+02,-9.999900e+02,...,-9.999900e+02,2.530000e+01,8.750000e+01,2.590000e+01,0.000000e+00,4.300000e+02,1.988000e+03,9.200000e+01,-9.999900e+02,3.650000e+00
max,-9.999900e+02,-9.999900e+02,-9.999900e+02,-9.999900e+02,8.441000e+01,1.084970e+03,1.119660e+03,2.740000e+02,-9.999900e+02,-9.999900e+02,...,-9.999900e+02,1.243000e+02,9.990000e+01,1.239000e+02,1.260000e+01,1.960000e+03,1.405200e+04,9.840000e+02,0.000000e+00,9.998000e+01


In [84]:
print(barbados_processed_df.info())
print(barbados_raw_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 30293 entries, 2020-06-08 15:00:00 to 2023-11-21 23:00:00
Data columns (total 23 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   bmp180_temp  0 non-null      float64
 1   bmp180_pres  0 non-null      float64
 2   bmp180_slp   0 non-null      float64
 3   bmp180_alt   0 non-null      float64
 4   bmp280_temp  19275 non-null  float64
 5   bmp280_pres  19304 non-null  float64
 6   bmp280_slp   19304 non-null  float64
 7   bmp280_alt   0 non-null      float64
 8   bme_temp     0 non-null      float64
 9   bme_pres     0 non-null      float64
 10  bme_slp      0 non-null      float64
 11  bme_alt      0 non-null      float64
 12  bme_hum      0 non-null      float64
 13  htu_temp     19539 non-null  float64
 14  htu_hum      15417 non-null  float64
 15  mcp9808      17389 non-null  float64
 16  tipping      29008 non-null  float64
 17  vis_light    22612 non-null  float64
 18  ir_light     21491 

In [85]:
counting_tipping_values(barbados_raw_df, "raw")

Raw or processed: raw
Tipping equals 0.0: 1721851
Tipping is non-zero and not NaN: 95657
Total number of tipping measurements (non-NaN): 1817508
Percentage of tipping values equal to 0.0: 94.74%
Percentage of tipping values non-zero: 5.26%


In [86]:
counting_tipping_values(barbados_processed_df, "processed")

Raw or processed: processed
Tipping equals 0.0: 26140
Tipping is non-zero and not NaN: 2868
Total number of tipping measurements (non-NaN): 29008
Percentage of tipping values equal to 0.0: 90.11%
Percentage of tipping values non-zero: 9.89%
